In [1]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
#from sympy import *
%load_ext cython
import time
import array
import numpy as np
import pythran


T 108.33078115826872


In [2]:

%load_ext pythran.magic

In [3]:
%%pythran
import numpy as np
#pythran export foo(float64[],float)
def foo(a,t): 
    rp=a.reshape((2,3,3))
    r=rp[0,:,:]
    p=rp[1,:,:]
    reserved=np.zeros((2,3,3),dtype=np.float64)
    reserved[0,:,:]=p
    reserved[1,:,:]=r
    return reserved

In [4]:
tst=np.array(['x', 'y', 'z', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 
              'px', 'py', 'pz', 'p_1x', 'p_1y', 'p_1z', 'p_2x', 'p_2y', 'p_2z'])
print(tst[0:3],'\n')
print(tst[3:6],'\n')
tst=tst.reshape((2,3,3))
print("\n",tst,"\n")
plus=[2,0,1]
minus=[1,2,0]
tstPlus=tst[:,plus,:]
tstMinus=tst[:,minus,:]
#et =[-1.0,1.0,1.0]
#e2t = [1,:,3]
print(tst[0,:,:],'\n',tstPlus[0,:,:],'\n',tstMinus[0,:,:],'\n',tstMinus[1,:,2])



['x' 'y' 'z'] 

['x_1' 'y_1' 'z_1'] 


 [[['x' 'y' 'z']
  ['x_1' 'y_1' 'z_1']
  ['x_2' 'y_2' 'z_2']]

 [['px' 'py' 'pz']
  ['p_1x' 'p_1y' 'p_1z']
  ['p_2x' 'p_2y' 'p_2z']]] 

[['x' 'y' 'z']
 ['x_1' 'y_1' 'z_1']
 ['x_2' 'y_2' 'z_2']] 
 [['x_2' 'y_2' 'z_2']
 ['x' 'y' 'z']
 ['x_1' 'y_1' 'z_1']] 
 [['x_1' 'y_1' 'z_1']
 ['x_2' 'y_2' 'z_2']
 ['x' 'y' 'z']] 
 ['p_1z' 'p_2z' 'pz']


In [6]:
import numpy as np
m_p=mpar.m_p
aep=mpar.aep
app=mpar.app
M=np.array([[1.0],[m_p],[m_p]])

PPlus  = np.array([2,0,1])
PMinus = np.array([1,2,0])
a2=np.array([[aep**2],[aep**2],[app**2]])
fc1=np.array([1,-1,1]).reshape(3,1)
fc2=np.array([-1,-1,1]).reshape(3,1)
global rhs
rhs=np.zeros((2,3,3),dtype=np.float64)

def  dif(a , t) :
    #global rhs
    rp=a.reshape((2,3,3))
    r=rp[0,:,:]
    p=rp[1,:,:]
    dr=r-r[PPlus,:]
    fplus_norm=np.sqrt(np.linalg.norm(dr,axis=1,keepdims=True)**2+a2)**3
    fplus=dr/fplus_norm
    
    ftotal=fplus*fc2 + fplus[PMinus,:]*fc1 
    rhs[0,:,:]=p/M
    rhs[1,:,:]=ftotal 
    return rhs.flatten()



In [ ]:
def  dif_v2(a , t) :
    #x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    rp=a.reshape((2,3,3))
    #print(rp)
    r=rp[0,:,:]
    p=rp[1,:,:]
    dr=r-r[PPlus,:]
    #print("dr=",dr)
    fplusn=np.sqrt(np.linalg.norm(rp[0,:,:]-rp[0,PPlus,:],axis=1,keepdims=True)**2+a2)**3
    #print("fplusn=",fplusn)
    fplus=(rp[0,:,:]-rp[0,PPlus,:])/fplusn
    #print(fplus)
    #print()
    ftotal=fplus[PMinus,:]*fc1+fplus*fc2
    #print(ftotal)
    rhs=np.zeros((2,3,3),dtype=np.float64)
    rhs[0,:,:]=p/M
    rhs[1:,:]=ftotal
    return rhs.flatten()

In [ ]:
a_0 =  np.array([0.0, 0.0, 0.0, 
                 0.0, 0.0, 1.0, 
                 0.0, 0.0, -1.0, 
                 0.0, 0.0, 0.0, 0.1, 0.2, 0.3, -0.1, -0.2, -0.3])
t=0.0
%time (dif(a_0,t))

In [8]:
#global rhs
#rhs=np.zeros((2,3,3),dtype=np.float64)
et = np.array([-1,1,1])
def E(t):
    E = mpar.E_0 * (np.sin(np.pi * t / mpar.T)) ** 2 * np.sin(mpar.w_0 * t)
    return E

def  dif2(a , t) :
    #global rhs
    rp=a.reshape((2,3,3))
    r=rp[0,:,:]
    p=rp[1,:,:]
    dr=r-r[PPlus,:]
    fplus_norm=np.sqrt(np.linalg.norm(dr,axis=1,keepdims=True)**2+a2)**3
    fplus=dr/fplus_norm
    
    ftotal=fplus*fc2 + fplus[PMinus,:]*fc1 
    rhs[0,:,:]=p/M
    rhs[1:,:]=ftotal
    rhs[1,:,2] += E(t)*et
    return rhs.flatten()




In [9]:
a_0 =  np.array([0.0, 0.0, 0.0, 
                 0.0, 0.0, 1.0, 
                 0.0, 0.0, -1.0, 
                 0.0, 0.0, 0.0, 0.1, 0.2, 0.3, -0.1, -0.2, -0.3])
t=1.0
%time (dif2(a_0,t))

CPU times: user 158 µs, sys: 59 µs, total: 217 µs
Wall time: 193 µs


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.44602984e-05,
        1.08920597e-04,  1.63380895e-04, -5.44602984e-05, -1.08920597e-04,
       -1.63380895e-04,  0.00000000e+00,  0.00000000e+00, -3.24994253e-06,
       -0.00000000e+00, -0.00000000e+00, -8.69221371e-02,  0.00000000e+00,
        0.00000000e+00,  8.69286370e-02])

In [12]:
%%timeit
d=dif(a_0,t)

26.5 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [13]:
%%timeit
d=dif2(a_0,t)

36.1 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
p = mpar.p
R_1 = np.array([-mpar.dist/2, 0.0, 0.0]) #начальные положения pr1,pr2,e
R_2 = np.array([mpar.dist/2, 0.0, 0.0])
r = np.array([0,0,0])

print (mpar.dist)
def a_0(i):
    a_0 = np.array([0.0, 0.0, 0.0,
                    -mpar.dist / 2, 0.0, 0.0,
                    mpar.dist / 2, 0.0, 0.0,
                    p[i][0], p[i][1], p[i][2], 
                    0.0, 0.0, 0.0, 
                    0.0, 0.0, 0.0])
    return a_0
solution =  []
start = time.time()
#odeint(dif, a_0(i), mpar.t)
#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))



#Решаем сиситему 18 диф. уравнений с лазером
t2 = np.linspace(mpar.u, mpar.u + 325, 100000)

def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
solution2 = []


# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

1.5070016255625132
Время выполнения: 16.944705963134766
Среднее время выполнения одного эксперимента: 1.6944705963134765
